In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

In [2]:
# --- Data Loading and Preprocessing ---
dataset1 = pd.read_csv("Wine.csv", index_col=None)
df2 = dataset1

In [3]:
# Convert all columns to a numeric type, forcing errors to become NaN
for column in df2.columns:
    df2[column] = pd.to_numeric(df2[column], errors='coerce')

In [4]:
# Handle NaN values by imputing with the mean of each column
for column in df2.columns:
    if df2[column].isnull().any():
        df2[column] = df2[column].fillna(df2[column].mean())

In [5]:
# Replace infinity values with NaN, then impute them
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
for column in df2.columns:
    if df2[column].isnull().any():
        df2[column] = df2[column].fillna(df2[column].mean())

In [6]:
# Split the data into independent and dependent variables
# Assuming 'Customer_Segment' is the target column
indep_X = df2.drop('Customer_Segment', axis=1).values
dep_Y = df2['Customer_Segment'].values

In [7]:
# --- Function Definitions ---
def split_scalar(indep_X, dep_Y):
    """Splits data into training and testing sets and scales them."""
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

In [8]:
def cm_prediction(classifier, X_test, y_test):
    """Makes predictions, calculates metrics, and returns them."""
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred) 
    Accuracy = accuracy_score(y_test, y_pred)
    # Return Accuracy, report, and confusion matrix
    return classifier, Accuracy, report, cm

In [9]:
def logistic(X_train, y_train, X_test, y_test):
    # Fitting Logistic Regression to the Training set
    classifier = LogisticRegression(random_state=0) # Added random_state for reproducibility
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

In [10]:
def svm_linear(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='linear', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

In [11]:
def svm_NL(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='rbf', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

In [12]:
def Navie(X_train, y_train, X_test, y_test):
    # Fitting GaussianNB to the training set
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

In [13]:
def knn(X_train, y_train, X_test, y_test):
    # Fitting K-NN to the training set
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

In [14]:
def Decision(X_train, y_train, X_test, y_test):
    # Fitting Decision Tree to the Training set
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

In [15]:
def random(X_train, y_train, X_test, y_test):
    # Fitting Random Forest to the Training set
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

In [16]:
def LDA_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf):
    dataframe = pd.DataFrame(index=['LDA'], columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number, idex in enumerate(dataframe.index):
        # Index is 'LDA', so number will be 0.
        dataframe.loc['LDA','Logistic'] = acclog[number]
        dataframe.loc['LDA','SVMl'] = accsvml[number]
        dataframe.loc['LDA','SVMnl'] = accsvmnl[number]
        dataframe.loc['LDA','KNN'] = accknn[number]
        dataframe.loc['LDA','Navie'] = accnav[number]
        dataframe.loc['LDA','Decision'] = accdes[number]
        dataframe.loc['LDA','Random'] = accrf[number]
    return dataframe

In [17]:
# --- Main Execution Flow (Corrected) ---

# 1. Split and Scale the cleaned data
X_train, X_test, y_train, y_test = split_scalar(indep_X, dep_Y)

In [18]:
# 2. Apply LDA
lda_components_to_test = [1, 2]
lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

In [19]:
# Use the LDA transformed data for classification
X_train_final = X_train_lda
X_test_final = X_test_lda

In [20]:
# Initialise lists to store accuracies
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [21]:
# Running the models on the LDA transformed data
_, Accuracy, _, _ = logistic(X_train_final, y_train, X_test_final, y_test)
acclog.append(Accuracy)

_, Accuracy, _, _ = svm_linear(X_train_final, y_train, X_test_final, y_test)
accsvml.append(Accuracy)

_, Accuracy, _, _ = svm_NL(X_train_final, y_train, X_test_final, y_test)
accsvmnl.append(Accuracy)

_, Accuracy, _, _ = knn(X_train_final, y_train, X_test_final, y_test)
accknn.append(Accuracy)

_, Accuracy, _, _ = Navie(X_train_final, y_train, X_test_final, y_test)
accnav.append(Accuracy)

_, Accuracy, _, _ = Decision(X_train_final, y_train, X_test_final, y_test)
accdes.append(Accuracy)

_, Accuracy, _, _ = random(X_train_final, y_train, X_test_final, y_test)
accrf.append(Accuracy)

# Calling the classification summary function (renamed from PCA_Classification)
result = LDA_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
LDA,1.0,0.977778,1.0,0.977778,0.977778,0.977778,0.977778
